In [ ]:
#| default_exp ml_classification

# Water pixels classification

Steps :
1. Collect water pixels into table (consider negative buffer for mroe accuracy)  
2. consider - filter pixels based on the values I found in task 2  
3. create dataframe  
4. split x,y,train,test and train model  
5. evaluate the model

In [1]:
#|hide
from nbdev.showdoc import *

In [2]:
#|export
#|hide

import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# import geopandas as gpd
# import matplotlib.pyplot as plt


from flood_exercise import utils_func
from flood_exercise import const_vals as CONST

C:\Users\USER\AppData\Local\Temp\ipykernel_11076\3491729059.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
#|export

class classification_pixels():
  
  def __init__(self,
               path_labels_str : str , #path to the labeled images
               path_imgs_str : str , #path to S2 images to be used for train
               target_col : str , #name of the target attribute in the dataframe. Will be used for model training
               cols_to_drop : list = [None] , # list of columns to be dropped from the dataframe. These columns won't be used to train the model. 
               test_size : float=CONST.TEST_SIZE , #size of test data
               random_state : int = CONST.RANDOM_STATE , #random state , deafult value is set in Cconst_vals module
               ):
    
    self.target_col = target_col
    self.cols_to_drop = cols_to_drop
    self.test_size = test_size
    self.random_state = random_state
    
    self.path_labels = utils_func.load_list_paths(path = path_labels_str , filter_file = True)
    self.path_imgs = utils_func.load_list_paths(path = path_imgs_str , filter_file= True)


    #collect pixels into dataframe 
    self.df_res = self._collect_pixels_to_dataframe_()

    # split into train and test data for ML model
    self.x_train, self.x_test, self.y_train, self.y_test=self._prepare_dataframe_for_train_()




  def _collect_pixels_to_dataframe_(self):
    dfs_pixels = []

    #TODO remove the 3!
    for label_path in self.path_labels[:3]:
      id_path = label_path.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[1]
      
      #find the matching S2 image
      s2_img_path = [x for x in self.path_imgs if id_path in x]
      
      if len(s2_img_path)==1:

        s2_img = rasterio.open(s2_img_path[0]).read()
        cols = rasterio.open(s2_img_path[0]).descriptions + rasterio.open(label_path).descriptions
        cols = [str(x) for x in cols]
        label_img = rasterio.open(label_path).read()

        stacked_img = np.concatenate((s2_img, label_img), axis=0)
        
        df_pixels = pd.DataFrame(stacked_img.reshape([stacked_img.shape[0],-1]).T)
        df_pixels.columns = cols
        dfs_pixels.append(df_pixels)

      else:
        continue
      
      
      df_res = pd.concat(dfs_pixels)

      return df_res
    
  
  def _prepare_dataframe_for_train_(self):
    
    #drop columns that are not relevant for the training (defined by user)
    
    if self.cols_to_drop[0] != None:

      try:
        self.df_res=self.df_res.drop(self.cols_to_drop,axis=1)

      except Exception as e:
        print(f'Could not drop the columns {self.cols_to_drop} with error:{e}')
        

    #drop null values
    self.df_res.dropna(axis=0,inplace=True)

    x = self.df_res.drop(self.target_col,axis=1)
    y = self.df_res[self.target_col].values
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size = self.test_size, 
                                                        random_state = self.random_state)

    return x_train, x_test, y_train, y_test
    


    

In [6]:
instance = classification_pixels(path_imgs_str=r"D:\git\flood_exercise\S2",
                                 path_labels_str=r"D:\git\flood_exercise\S2_HANDLABELED",
                                 target_col = 'None',
                                 cols_to_drop= ['qc'],
                                 test_size= 0.25,
                                 random_state = 42,
                    
                                 )
instance.df_res

Could not drop the columns ['qc'] with error:"['qc'] not found in axis"
I have x and y


,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12,None
0,1518,1233,1186,875,1256,2642,3335,3061,3561,342,35,2394,1158,0
1,1518,1239,1194,897,1256,2642,3335,2887,3561,342,35,2394,1158,0
2,1517,1251,1253,933,1381,2806,3312,3115,3806,360,33,2563,1387,0
3,1517,1255,1247,936,1381,2806,3312,3181,3806,360,33,2563,1387,0
4,1517,1228,1197,897,1288,2498,3048,2898,3362,360,33,2493,1375,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262139,1681,1392,1390,1082,1468,2821,3442,3195,3831,481,133,2374,1172,0
262140,1681,1390,1379,1099,1468,2821,3442,3044,3831,481,133,2374,1172,0
262141,1681,1373,1372,1086,1439,2680,3159,2991,3537,481,133,2318,1195,0
262142,1681,1363,1335,1060,1439,2680,3159,2917,3537,481,133,2318,1195,0


In [8]:
instance.x

,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12
0,1518,1233,1186,875,1256,2642,3335,3061,3561,342,35,2394,1158
1,1518,1239,1194,897,1256,2642,3335,2887,3561,342,35,2394,1158
2,1517,1251,1253,933,1381,2806,3312,3115,3806,360,33,2563,1387
3,1517,1255,1247,936,1381,2806,3312,3181,3806,360,33,2563,1387
4,1517,1228,1197,897,1288,2498,3048,2898,3362,360,33,2493,1375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262139,1681,1392,1390,1082,1468,2821,3442,3195,3831,481,133,2374,1172
262140,1681,1390,1379,1099,1468,2821,3442,3044,3831,481,133,2374,1172
262141,1681,1373,1372,1086,1439,2680,3159,2991,3537,481,133,2318,1195
262142,1681,1363,1335,1060,1439,2680,3159,2917,3537,481,133,2318,1195


In [ ]:
instance.x

In [ ]:
instance.df_res.columns

In [ ]:
instance.df_res.drop('None',axis=1,inplace=False)

In [ ]:
instance.df_res['None'].unique().tolist()

In [ ]:
instance.df_res

In [ ]:
pd.DataFrame(test.reshape([test.shape[0],-1]).T)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(test[13,:,:])